In [1]:
import sys
sys.path.append("../scr/utils")

import os
import yaml
import time
import datetime
import warnings
import platform
from tqdm import tqdm

import torch
import timm
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader

from create_dataset import BirdDataset
from base_utils import set_seed
from metrics import validation_epoch_end

In [2]:
# Информация о железе, на котором тестируется модель

processor_info = platform.processor()
print("Информация о процессоре:", processor_info)

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print("Доступно GPU:", num_gpus)
    for i in range(num_gpus):
        gpu = torch.cuda.get_device_name(i)
        print(f"GPU {i + 1}: {gpu}")   
else:
    print("GPU недоступны на данной системе.")

Информация о процессоре: Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
Доступно GPU: 1
GPU 1: NVIDIA GeForce GTX 1070 Ti


In [3]:
df = pd.read_csv("../data/data.csv")
df_test = df[df.fold == 3].reset_index(drop=True)  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_base = torch.load("../experiment/14_September_2023_16_29/model_tf_efficientnet_b0_last_version.pt").to(device)
dataset_test = BirdDataset(df=df_test,
                           path_to_folder_with_audio="../data")
valid_loader = DataLoader(dataset_test,
                          batch_size=4)
model_base.eval()
device

device(type='cuda')

Базовая модель

In [4]:
model = model_base

In [5]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 16.738MB


Проверим работу модели, при переводе модели из fp32 -> fp16 

In [6]:
model = model.half()

In [7]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 8.369MB


In [8]:
# Проверим точность модели при запуске на нашем валидационном датасете
predicted_labels_list = None
true_labels_list = None
metric = validation_epoch_end
start_time = time.time()


with torch.no_grad():
    for batch in tqdm(valid_loader):
        X_batch = batch[0].to(torch.float16).to(device)
        y_batch = batch[1].to(device)
        res = model.forward(X_batch)

        res = res.detach().sigmoid().cpu().numpy()
        y_batch_onehot = y_batch
        y_batch_onehot = y_batch_onehot.unsqueeze(1).detach().cpu().numpy()
        y_batch_onehot = y_batch_onehot.squeeze()

        if predicted_labels_list is None:
            predicted_labels_list = res
            true_labels_list = y_batch_onehot
        else:
            predicted_labels_list = np.concatenate([predicted_labels_list, res], axis=0)
            true_labels_list = np.concatenate([true_labels_list, y_batch_onehot], axis=0)      

        del batch, res
    torch.cuda.empty_cache()

end_time = time.time()

100%|██████████| 771/771 [03:31<00:00,  3.65it/s]


In [9]:
all_predicted_labels = np.vstack(predicted_labels_list)
all_true_labels = np.vstack(true_labels_list)
all_true_labels = np.squeeze(all_true_labels)
mask = (all_true_labels > 0) & (all_true_labels < 1)
all_true_labels[mask] = 0
avg_metric = metric(all_true_labels, all_predicted_labels)

In [10]:
t = end_time - start_time

In [11]:
# Мы понимаем, что ко времени работы модели добавляем время обработки батчей и добавления аугментаций к стартовым данным

print("Наши метрики на model.half()")
print(f"Время работы модели на всем батче {t:.<2g} сек.")
print(f"Время работы модели на одном сэмпле (AVG) {round(t/ len(dataset_test), 3) } сек.")
print("Метрики качества")
for m in avg_metric:
    print(f"metric {m} : {avg_metric[m]:.<5g}")

Наши метрики на model.half()
Время работы модели на всем батче 211.396 сек.
Время работы модели на одном сэмпле (AVG) 0.069 сек.
Метрики качества
metric val_RMAP : 0.035317
metric CMAP_5 : 0.0204819


Вес модели уменьшился, но метрики просели очень сильно. Данный способ компрессии модели нам не подходит. См. блокнот hw_2_2